### 重叠政策不同阈值簇聚类图

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import seaborn as sns
from scipy.cluster.hierarchy import linkage, fcluster, leaves_list
from pathlib import Path
import colorsys
import warnings
from joblib import Parallel, delayed  # ✅ 重新引入并行

# 禁用警告
warnings.filterwarnings('ignore')

# ==========================================
# 1. 基础配置
# ==========================================
current_dir = Path.cwd()
data_dir = current_dir.parent / "data"
base_output_dir = data_dir / "4-2-group_overlapping_cluster"
base_output_dir.mkdir(parents=True, exist_ok=True)

def get_times_black_font():
    return FontProperties(family='Times New Roman', size=26, weight='black')

T_BLACK = get_times_black_font()

plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Times New Roman'],
    'font.weight': 'black',
    'axes.labelweight': 'black',
    'axes.titleweight': 'black',
    'axes.unicode_minus': False,
    'figure.dpi': 300
})

sns.set_theme(style="white")
plt.rcParams['axes.axisbelow'] = False

def lighten_color_slightly(hex_color: str) -> str:
    r, g, b = plt.cm.colors.to_rgb(hex_color)
    h, l, s = colorsys.rgb_to_hls(r, g, b)
    return plt.cm.colors.to_hex(colorsys.hls_to_rgb(h, min(1.0, l + 0.12), s * 0.88))

def get_palette(k: int) -> list:
    colors = ['#1f77b4', '#2ca02c', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
    return (colors * (k // len(colors) + 1))[:k]

# ==========================================
# 2. 数据处理
# ==========================================
def build_overlap_matrix(input_file: Path) -> tuple[pd.DataFrame, int]:
    if not input_file.exists():
        return None, 0

    df = pd.read_csv(input_file, encoding='utf-8-sig')

    if 'L2政策' not in df.columns and 'L2政策中文名' in df.columns:
        df.rename(columns={'L2政策中文名': 'L2政策'}, inplace=True)

    def safe_col(d, name):
        return d[name].iloc[:, 0] if isinstance(d[name], pd.DataFrame) else d[name]

    df_clean = pd.DataFrame({
        'P': safe_col(df, 'L2政策').astype(str),
        'C': safe_col(df, '国家').astype(str),
        'G': safe_col(df, '聚类ID')
    }).drop_duplicates()

    policies = sorted(df_clean['P'].unique())
    countries = sorted(df_clean['C'].unique())

    res_map = {
        p: dict(zip(
            df_clean[df_clean['P'] == p]['C'],
            df_clean[df_clean['P'] == p]['G']
        )) for p in policies
    }

    n = len(countries)
    matrix = np.zeros((n, n), dtype=int)

    for i, c1 in enumerate(countries):
        for j in range(i, n):
            c2 = countries[j]
            common = sum(
                1 for p in policies
                if c1 in res_map[p]
                and c2 in res_map[p]
                and (res_map[p][c1] == res_map[p][c2] if i != j else True)
            )
            matrix[i, j] = matrix[j, i] = common

    return pd.DataFrame(matrix, index=countries, columns=countries), len(policies)

# ==========================================
# 3. 核心绘图 (独立函数，供并行调用)
# ==========================================
def draw_and_save_k(
    k: int,
    Z: np.ndarray,
    df_o: pd.DataFrame,
    max_o: int,
    output_folder: Path,
    type_name: str
) -> str:
    """
    单个K值的处理逻辑：生成颜色 -> 导出CSV -> 绘图
    """
    try:
        # 1. 计算标签
        labels = fcluster(Z, t=k, criterion='maxclust') - 1
        palette = get_palette(k)
        row_colors = pd.Series(
            [lighten_color_slightly(palette[l % len(palette)]) for l in labels],
            index=df_o.index
        )

        # 2. 导出 CSV
        csv_filename = f'{type_name}_cluster_k={k}.csv'
        groups = {}
        for country, cid in zip(df_o.index.tolist(), labels.astype(int).tolist()):
            groups.setdefault(cid, []).append(country)

        rows = []
        for cid in sorted(groups):
            members = groups[cid]
            rows.append({
                '共识ID': cid + 1,
                '配色HEX': lighten_color_slightly(palette[cid % len(palette)]),
                '国家数': len(members),
                '国家列表': ', '.join(sorted(members)),
                '簇内平均重叠': f'{df_o.loc[members, members].values.mean():.2f}'
            })
        
        pd.DataFrame(rows).to_csv(output_folder / csv_filename, index=False, encoding='utf-8-sig')

        # 3. 绘图 (严格 Style Reference)
        png_filename = f'{type_name}_cluster_k={k}.png'
        size = max(18, len(df_o) * 0.72)

        g = sns.clustermap(
            df_o,
            row_linkage=Z,
            col_linkage=Z,
            cmap='RdYlBu_r',
            row_colors=row_colors,
            col_colors=row_colors,
            dendrogram_ratio=0.1,
            linewidths=0.5,
            figsize=(size, size),
            annot=True,
            fmt='d',
            vmin=0,
            vmax=max_o,
            annot_kws={'fontsize': 28, 'weight': 'black', 'fontproperties': T_BLACK},
            cbar_kws={'ticks': [0, int(max_o*0.25), int(max_o*0.5), int(max_o*0.75), max_o]},
            tree_kws={'linewidths': 6}
        )

        # 调整 Colorbar 位置
        hm_pos = g.ax_heatmap.get_position()
        pad = 0.04; cbar_w = 0.018; cbar_h = hm_pos.height * 0.78
        cbar_y = hm_pos.y0 + hm_pos.height * 0.11
        g.cax.set_position([hm_pos.x1 + pad, cbar_y, cbar_w, cbar_h])
        g.cax.yaxis.set_label_position('right')

        g.cax.set_axisbelow(False)
        for coll in g.cax.collections: coll.set_zorder(0)
        g.cax.yaxis.set_ticks_position('both')
        g.cax.tick_params(axis='y', direction='in', length=12, width=3, colors='black', left=True, right=True, pad=6)

        for t in g.cax.yaxis.get_major_ticks():
            for ln in (t.tick1line, t.tick2line):
                ln.set_zorder(10); ln.set_clip_on(False)
        for lab in g.cax.get_yticklabels(): lab.set_zorder(11)

        g.cax.set_ylabel(f'Overlap Count ({type_name})', fontproperties=T_BLACK, fontsize=32, weight='black', labelpad=20)
        plt.setp(g.cax.get_yticklabels(), fontproperties=T_BLACK, fontsize=28, weight='black')

        # 轴标签
        for lab in g.ax_heatmap.get_xticklabels():
            lab.set_fontproperties(T_BLACK); lab.set_rotation(45); lab.set_ha('right')
        for lab in g.ax_heatmap.get_yticklabels():
            lab.set_fontproperties(T_BLACK); lab.set_rotation(0)

        # 粗边框
        ax = g.ax_heatmap
        for spine in ax.spines.values():
            spine.set_visible(True); spine.set_linewidth(4); spine.set_edgecolor('black')

        # 绘制聚类黑框
        order = g.dendrogram_row.reordered_ind
        ordered_labels = labels[order].astype(int).tolist()
        ranges = {}
        for idx, cid in enumerate(ordered_labels):
            if cid not in ranges: ranges[cid] = (idx, idx)
            else: ranges[cid] = (ranges[cid][0], idx)

        boundary_color = '#4D4D4D'
        for _, (s, e) in ranges.items():
            ax.plot([s, s], [s, e + 1], color=boundary_color, linewidth=6, clip_on=False, zorder=20)
            ax.plot([e + 1, e + 1], [s, e + 1], color=boundary_color, linewidth=6, clip_on=False, zorder=20)
            ax.plot([s, e + 1], [s, s], color=boundary_color, linewidth=6, clip_on=False, zorder=20)
            ax.plot([s, e + 1], [e + 1, e + 1], color=boundary_color, linewidth=6, clip_on=False, zorder=20)

        # 保存
        plt.savefig(output_folder / png_filename, dpi=300, bbox_inches='tight')
        plt.close('all') # 关键：关闭图形以释放内存
        
        return f"[{type_name}] k={k} 完成"

    except Exception as e:
        return f"[{type_name}] k={k} 失败: {str(e)}"

# ==========================================
# 4. 强制左偏排序 (Force Left Linkage)
# ==========================================
def compute_force_left_linkage(df: pd.DataFrame, type_name: str) -> np.ndarray:
    print(f"   正在计算聚类树并检查重心...")
    Z = linkage(df.values, method='ward')
    weights = df.sum(axis=1).values

    # 局部排序
    w_map = {i: v for i, v in enumerate(weights)}
    for i, (c1, c2, _, _) in enumerate(Z):
        c1, c2 = int(c1), int(c2)
        if w_map[c1] < w_map[c2]: 
            Z[i, 0], Z[i, 1] = c2, c1
        w_map[len(weights) + i] = w_map[c1] + w_map[c2]

    # 全局重心翻转
    leaves = leaves_list(Z)
    mid = len(leaves) // 2
    left_weight = weights[leaves[:mid]].sum()
    right_weight = weights[leaves[mid:]].sum()

    print(f"   [{type_name} 重心] 左: {left_weight:.0f} | 右: {right_weight:.0f}")

    if right_weight > left_weight:
        print(f"   >>> ⚠️ 检测到重心在右，执行镜像翻转...")
        Z[:, [0, 1]] = Z[:, [1, 0]]
    else:
        print(f"   >>> ✅ 重心正常。")
    
    return Z

def get_valid_k_values(Z: np.ndarray, n_items: int) -> list[int]:
    valid = []
    for k in range(2, min(30, n_items) + 1):
        labels = fcluster(Z, t=k, criterion='maxclust')
        if len(set(labels)) == k:
            valid.append(k)
            if len(valid) == 20:
                break
    return valid

# ==========================================
# 5. 主流程
# ==========================================
def process_type_pipeline(file_path: Path, type_name: str):
    """
    单个类型的主处理逻辑
    """
    print(f"\n============================================")
    print(f" 🚀 开始处理: {type_name}")
    print(f"============================================")

    # 1. 准备文件夹
    # ✅ 关键修改：为每个类型创建一个子文件夹
    type_output_dir = base_output_dir / type_name
    type_output_dir.mkdir(parents=True, exist_ok=True)
    print(f"   📂 结果将保存至: {type_output_dir}")

    # 2. 读取数据
    df_o, max_o = build_overlap_matrix(file_path)
    if df_o is None: 
        print(f"   ❌ 跳过，文件未找到")
        return

    # 3. 计算 Linkage
    Z = compute_force_left_linkage(df_o, type_name)

    # 4. 获取有效 K 值
    valid_k = get_valid_k_values(Z, len(df_o))
    print(f"   🎯 待处理 K 值 ({len(valid_k)}个): {valid_k}")

    # 5. 并行执行绘图
    # ✅ 关键修改：并行处理
    results = Parallel(n_jobs=-3, verbose=5)(
        delayed(draw_and_save_k)(
            k, Z, df_o, max_o, type_output_dir, type_name
        ) for k in valid_k
    )

    for res in results:
        print(f"   {res}")

def main():
    # 定义任务列表
    files = {
        "Breadth": data_dir / "3-1-L2_Policy_Clustering_Breadth.csv",
        "Intensity": data_dir / "3-1-L2_Policy_Clustering_Intensity.csv"
    }

    for type_name, path in files.items():
        process_type_pipeline(path, type_name)

    print("\n✅ 所有任务已完成！")

if __name__ == "__main__":
    main()


 🚀 开始处理: Breadth
   📂 结果将保存至: f:\Desktop\科研项目\1.负责科研项目\Climate Policy\CAMPF_Supplementary\data\4-2-group_overlapping_cluster\Breadth
   正在计算聚类树并检查重心...
   [Breadth 重心] 左: 6977 | 右: 8088
   >>> ⚠️ 检测到重心在右，执行镜像翻转...
   🎯 待处理 K 值 (20个): [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]


[Parallel(n_jobs=-3)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=-3)]: Done   6 out of  20 | elapsed:  1.9min remaining:  4.4min
[Parallel(n_jobs=-3)]: Done  11 out of  20 | elapsed:  2.0min remaining:  1.6min
[Parallel(n_jobs=-3)]: Done  16 out of  20 | elapsed:  2.0min remaining:   30.0s
[Parallel(n_jobs=-3)]: Done  20 out of  20 | elapsed:  2.1min finished


   [Breadth] k=2 完成
   [Breadth] k=3 完成
   [Breadth] k=4 完成
   [Breadth] k=5 完成
   [Breadth] k=6 完成
   [Breadth] k=7 完成
   [Breadth] k=8 完成
   [Breadth] k=9 完成
   [Breadth] k=10 完成
   [Breadth] k=11 完成
   [Breadth] k=12 完成
   [Breadth] k=13 完成
   [Breadth] k=14 完成
   [Breadth] k=15 完成
   [Breadth] k=16 完成
   [Breadth] k=17 完成
   [Breadth] k=18 完成
   [Breadth] k=19 完成
   [Breadth] k=20 完成
   [Breadth] k=21 完成

 🚀 开始处理: Intensity
   📂 结果将保存至: f:\Desktop\科研项目\1.负责科研项目\Climate Policy\CAMPF_Supplementary\data\4-2-group_overlapping_cluster\Intensity
   正在计算聚类树并检查重心...
   [Intensity 重心] 左: 8066 | 右: 7731
   >>> ✅ 重心正常。
   🎯 待处理 K 值 (20个): [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]


[Parallel(n_jobs=-3)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=-3)]: Done   6 out of  20 | elapsed:  1.3min remaining:  3.0min
[Parallel(n_jobs=-3)]: Done  11 out of  20 | elapsed:  1.3min remaining:  1.1min
[Parallel(n_jobs=-3)]: Done  16 out of  20 | elapsed:  1.3min remaining:   20.0s


   [Intensity] k=2 完成
   [Intensity] k=3 完成
   [Intensity] k=4 完成
   [Intensity] k=5 完成
   [Intensity] k=6 完成
   [Intensity] k=7 完成
   [Intensity] k=8 完成
   [Intensity] k=9 完成
   [Intensity] k=10 完成
   [Intensity] k=11 完成
   [Intensity] k=12 完成
   [Intensity] k=13 完成
   [Intensity] k=14 完成
   [Intensity] k=15 完成
   [Intensity] k=16 完成
   [Intensity] k=17 完成
   [Intensity] k=18 完成
   [Intensity] k=19 完成
   [Intensity] k=20 完成
   [Intensity] k=21 完成

✅ 所有任务已完成！


[Parallel(n_jobs=-3)]: Done  20 out of  20 | elapsed:  1.3min finished


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import seaborn as sns
from scipy.cluster.hierarchy import linkage, fcluster, leaves_list
from scipy.spatial.distance import squareform
from pathlib import Path
import colorsys
import warnings
from joblib import Parallel, delayed  # ✅ 并行

# 禁用警告
warnings.filterwarnings('ignore')

# ==========================================
# 1. 基础配置
# ==========================================
current_dir = Path.cwd()
data_dir = current_dir.parent / "data"
base_output_dir = data_dir / "4-2-group_overlapping_cluster"
base_output_dir.mkdir(parents=True, exist_ok=True)

def get_times_black_font():
    return FontProperties(family='Times New Roman', size=26, weight='black')

T_BLACK = get_times_black_font()

plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Times New Roman'],
    'font.weight': 'black',
    'axes.labelweight': 'black',
    'axes.titleweight': 'black',
    'axes.unicode_minus': False,
    'figure.dpi': 300
})

sns.set_theme(style="white")
plt.rcParams['axes.axisbelow'] = False

def lighten_color_slightly(hex_color: str) -> str:
    r, g, b = plt.cm.colors.to_rgb(hex_color)
    h, l, s = colorsys.rgb_to_hls(r, g, b)
    return plt.cm.colors.to_hex(colorsys.hls_to_rgb(h, min(1.0, l + 0.12), s * 0.88))

def get_palette(k: int) -> list:
    colors = ['#1f77b4', '#2ca02c', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
    return (colors * (k // len(colors) + 1))[:k]

# ✅ 左开右闭阈值：Overlap ∈ (t-1, +∞]，对整数 overlap 等价于 Overlap >= t
THRESHOLDS_RC = list(range(5, 16))  # 5..15

# ==========================================
# 2. 数据处理（保持不变：共识矩阵=重叠次数）
# ==========================================
def build_overlap_matrix(input_file: Path) -> tuple[pd.DataFrame, int]:
    if not input_file.exists():
        return None, 0

    df = pd.read_csv(input_file, encoding='utf-8-sig')

    if 'L2政策' not in df.columns and 'L2政策中文名' in df.columns:
        df.rename(columns={'L2政策中文名': 'L2政策'}, inplace=True)

    def safe_col(d, name):
        return d[name].iloc[:, 0] if isinstance(d[name], pd.DataFrame) else d[name]

    df_clean = pd.DataFrame({
        'P': safe_col(df, 'L2政策').astype(str),
        'C': safe_col(df, '国家').astype(str),
        'G': safe_col(df, '聚类ID')
    }).drop_duplicates()

    policies = sorted(df_clean['P'].unique())
    countries = sorted(df_clean['C'].unique())

    res_map = {
        p: dict(zip(
            df_clean[df_clean['P'] == p]['C'],
            df_clean[df_clean['P'] == p]['G']
        )) for p in policies
    }

    n = len(countries)
    matrix = np.zeros((n, n), dtype=int)

    for i, c1 in enumerate(countries):
        for j in range(i, n):
            c2 = countries[j]
            common = sum(
                1 for p in policies
                if c1 in res_map[p]
                and c2 in res_map[p]
                and (res_map[p][c1] == res_map[p][c2] if i != j else True)
            )
            matrix[i, j] = matrix[j, i] = common

    return pd.DataFrame(matrix, index=countries, columns=countries), len(policies)

# ==========================================
# 3. 核心绘图（可视化不变）- 改为“阈值切割”
#    ✅ 改动点：阈值采用 左开右闭 => Overlap >= t
# ==========================================
def draw_and_save_threshold(
    threshold_rc: int,
    Z: np.ndarray,
    df_o: pd.DataFrame,
    max_o: int,
    output_folder: Path,
    type_name: str
) -> str:
    """
    单个阈值处理逻辑：阈值切割(Complete Linkage) -> 颜色 -> 导出CSV -> 绘图

    ✅ 左开右闭（对整数 overlap）：Overlap ∈ (t-1, +∞]  <=>  Overlap >= t
    """
    try:
        # ✅ 左开右闭：Overlap >= t
        min_overlap = threshold_rc

        # distance = max_o - overlap
        # overlap >= min_overlap  <=>  distance <= max_o - min_overlap
        d_cut = max_o - min_overlap

        # 1) 阈值切割：criterion='distance'
        n = df_o.shape[0]
        if min_overlap > max_o:
            # 不可能满足 overlap >= min_overlap：全部单点簇
            labels_raw = np.arange(1, n + 1, dtype=int)
        else:
            # SciPy 的 distance cut 是 <= t（右闭），符合“右闭”要求
            labels_raw = fcluster(Z, t=float(d_cut), criterion='distance')

        # 重新映射为 0..k-1（保证配色与边框逻辑稳定）
        uniq = sorted(set(labels_raw.tolist()))
        relabel = {old: i for i, old in enumerate(uniq)}
        labels = np.array([relabel[x] for x in labels_raw], dtype=int)
        k = len(uniq)

        palette = get_palette(max(k, 1))
        row_colors = pd.Series(
            [lighten_color_slightly(palette[l % len(palette)]) for l in labels],
            index=df_o.index
        )

        # 2) 导出 CSV（加入核心-边缘 + 阈值校验信息）
        csv_filename = f'{type_name}_consensus_threshold_rc={threshold_rc}.csv'

        groups = {}
        for country, cid in zip(df_o.index.tolist(), labels.astype(int).tolist()):
            groups.setdefault(cid, []).append(country)

        rows = []
        for cid in sorted(groups):
            members = sorted(groups[cid])
            sub = df_o.loc[members, members].values

            if len(members) > 1:
                off = sub[~np.eye(len(members), dtype=bool)]
                min_off = int(np.min(off))
                mean_off = float(np.mean(off))
                is_core = True
                # Complete Linkage + distance cut 理论上应满足：簇内任意两国 overlap >= min_overlap
                pass_check = (min_off >= min_overlap)
            else:
                min_off = np.nan
                mean_off = np.nan
                is_core = False
                pass_check = True  # 单点无“任意两国”约束

            rows.append({
                'Threshold条件': f'Overlap ∈ ({threshold_rc - 1}, +∞]  (等价: Overlap ≥ {threshold_rc})',
                '等价最小重叠': min_overlap,
                '共识ID': cid + 1,
                '核心-边缘': 'Core' if is_core else 'Periphery',
                '配色HEX': lighten_color_slightly(palette[cid % len(palette)]),
                '国家数': len(members),
                '国家列表': ', '.join(members),
                '簇内最小重叠(非对角)': min_off,
                '簇内平均重叠(非对角)': (f'{mean_off:.2f}' if len(members) > 1 else ''),
                '阈值校验(最小重叠>=等价门槛)': pass_check
            })

        pd.DataFrame(rows).to_csv(output_folder / csv_filename, index=False, encoding='utf-8-sig')

        # 3) 绘图（严格沿用你的 Style Reference，不改任何可视化参数）
        png_filename = f'{type_name}_consensus_threshold_rc={threshold_rc}.png'
        size = max(18, len(df_o) * 0.72)

        g = sns.clustermap(
            df_o,
            row_linkage=Z,
            col_linkage=Z,
            cmap='RdYlBu_r',
            row_colors=row_colors,
            col_colors=row_colors,
            dendrogram_ratio=0.1,
            linewidths=0.5,
            figsize=(size, size),
            annot=True,
            fmt='d',
            vmin=0,
            vmax=max_o,
            annot_kws={'fontsize': 28, 'weight': 'black', 'fontproperties': T_BLACK},
            cbar_kws={'ticks': [0, int(max_o*0.25), int(max_o*0.5), int(max_o*0.75), max_o]},
            tree_kws={'linewidths': 6}
        )

        # 调整 Colorbar 位置
        hm_pos = g.ax_heatmap.get_position()
        pad = 0.04; cbar_w = 0.018; cbar_h = hm_pos.height * 0.78
        cbar_y = hm_pos.y0 + hm_pos.height * 0.11
        g.cax.set_position([hm_pos.x1 + pad, cbar_y, cbar_w, cbar_h])
        g.cax.yaxis.set_label_position('right')

        g.cax.set_axisbelow(False)
        for coll in g.cax.collections: coll.set_zorder(0)
        g.cax.yaxis.set_ticks_position('both')
        g.cax.tick_params(axis='y', direction='in', length=12, width=3, colors='black', left=True, right=True, pad=6)

        for t in g.cax.yaxis.get_major_ticks():
            for ln in (t.tick1line, t.tick2line):
                ln.set_zorder(10); ln.set_clip_on(False)
        for lab in g.cax.get_yticklabels(): lab.set_zorder(11)

        g.cax.set_ylabel(f'Overlap Count ({type_name})', fontproperties=T_BLACK, fontsize=32, weight='black', labelpad=20)
        plt.setp(g.cax.get_yticklabels(), fontproperties=T_BLACK, fontsize=28, weight='black')

        # 轴标签
        for lab in g.ax_heatmap.get_xticklabels():
            lab.set_fontproperties(T_BLACK); lab.set_rotation(45); lab.set_ha('right')
        for lab in g.ax_heatmap.get_yticklabels():
            lab.set_fontproperties(T_BLACK); lab.set_rotation(0)

        # 粗边框
        ax = g.ax_heatmap
        for spine in ax.spines.values():
            spine.set_visible(True); spine.set_linewidth(4); spine.set_edgecolor('black')

        # 绘制聚类黑框（基于当前阈值标签）
        order = g.dendrogram_row.reordered_ind
        ordered_labels = labels[order].astype(int).tolist()
        ranges = {}
        for idx, cid in enumerate(ordered_labels):
            if cid not in ranges: ranges[cid] = (idx, idx)
            else: ranges[cid] = (ranges[cid][0], idx)

        boundary_color = '#4D4D4D'
        for _, (s, e) in ranges.items():
            ax.plot([s, s], [s, e + 1], color=boundary_color, linewidth=6, clip_on=False, zorder=20)
            ax.plot([e + 1, e + 1], [s, e + 1], color=boundary_color, linewidth=6, clip_on=False, zorder=20)
            ax.plot([s, e + 1], [s, s], color=boundary_color, linewidth=6, clip_on=False, zorder=20)
            ax.plot([s, e + 1], [e + 1, e + 1], color=boundary_color, linewidth=6, clip_on=False, zorder=20)

        plt.savefig(output_folder / png_filename, dpi=300, bbox_inches='tight')
        plt.close('all')

        return f"[{type_name}] RC阈值 t={threshold_rc} 完成 (簇数={k}, d_cut={d_cut})"

    except Exception as e:
        return f"[{type_name}] RC阈值 t={threshold_rc} 失败: {str(e)}"

# ==========================================
# 4. Complete Linkage + 距离转换 + 强制左偏排序
# ==========================================
def compute_force_left_linkage_complete(df_o: pd.DataFrame, max_o: int, type_name: str) -> np.ndarray:
    """
    关键改造点：
    - distance = max_o - overlap
    - linkage(method='complete')，保证簇内任意两点距离<=cut
      => 等价于 overlap >= max_o - cut
    """
    print(f"   正在计算聚类树（Complete Linkage）并检查重心...")

    dist = (max_o - df_o).astype(float)
    np.fill_diagonal(dist.values, 0.0)  # 距离矩阵对角强制为0
    condensed = squareform(dist.values, checks=False)

    Z = linkage(condensed, method='complete')

    weights = df_o.sum(axis=1).values

    # 局部排序（维持你原来的“左偏”风格）
    w_map = {i: v for i, v in enumerate(weights)}
    for i, (c1, c2, _, _) in enumerate(Z):
        c1, c2 = int(c1), int(c2)
        if w_map.get(c1, 0) < w_map.get(c2, 0):
            Z[i, 0], Z[i, 1] = c2, c1
        w_map[len(weights) + i] = w_map.get(c1, 0) + w_map.get(c2, 0)

    # 全局重心翻转
    leaves = leaves_list(Z)
    mid = len(leaves) // 2
    left_weight = weights[leaves[:mid]].sum()
    right_weight = weights[leaves[mid:]].sum()

    print(f"   [{type_name} 重心] 左: {left_weight:.0f} | 右: {right_weight:.0f}")

    if right_weight > left_weight:
        print(f"   >>> ⚠️ 检测到重心在右，执行镜像翻转...")
        Z[:, [0, 1]] = Z[:, [1, 0]]
    else:
        print(f"   >>> ✅ 重心正常。")

    return Z

# ==========================================
# 5. 主流程（阈值扫描：左开右闭）
# ==========================================
def process_type_pipeline(file_path: Path, type_name: str):
    print(f"\n============================================")
    print(f" 🚀 开始处理: {type_name}")
    print(f"============================================")

    type_output_dir = base_output_dir / type_name
    type_output_dir.mkdir(parents=True, exist_ok=True)
    print(f"   📂 结果将保存至: {type_output_dir}")

    df_o, max_o = build_overlap_matrix(file_path)
    if df_o is None:
        print(f"   ❌ 跳过，文件未找到")
        return

    # ✅ Complete Linkage + 距离转换
    Z = compute_force_left_linkage_complete(df_o, max_o, type_name)

    # ✅ 阈值列表（左开右闭）
    print(f"   🎯 待处理阈值RC ({len(THRESHOLDS_RC)}个): {THRESHOLDS_RC}")
    print(f"      解释：每个 t 表示 Overlap ∈ (t-1, +∞]（等价 Overlap ≥ t）")

    results = Parallel(n_jobs=-3, verbose=5)(
        delayed(draw_and_save_threshold)(
            t_rc, Z, df_o, max_o, type_output_dir, type_name
        ) for t_rc in THRESHOLDS_RC
    )

    for res in results:
        print(f"   {res}")

def main():
    files = {
        "Breadth": data_dir / "3-1-L2_Policy_Clustering_Breadth.csv",
        "Intensity": data_dir / "3-1-L2_Policy_Clustering_Intensity.csv"
    }

    for type_name, path in files.items():
        process_type_pipeline(path, type_name)

    print("\n✅ 所有任务已完成！")

if __name__ == "__main__":
    main()



 🚀 开始处理: Breadth
   📂 结果将保存至: f:\Desktop\科研项目\1.负责科研项目\Climate Policy\CAMPF_Supplementary\data\4-2-group_overlapping_cluster\Breadth
   正在计算聚类树（Complete Linkage）并检查重心...
   [Breadth 重心] 左: 7972 | 右: 7093
   >>> ✅ 重心正常。
   🎯 待处理阈值RC (11个): [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
      解释：每个 t 表示 Overlap ∈ (t-1, +∞]（等价 Overlap ≥ t）


[Parallel(n_jobs=-3)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=-3)]: Done   3 out of  11 | elapsed:   25.2s remaining:  1.1min
[Parallel(n_jobs=-3)]: Done   6 out of  11 | elapsed:   27.0s remaining:   22.5s
[Parallel(n_jobs=-3)]: Done   9 out of  11 | elapsed:   27.2s remaining:    6.0s
[Parallel(n_jobs=-3)]: Done  11 out of  11 | elapsed:   27.5s finished
[Parallel(n_jobs=-3)]: Using backend LokyBackend with 30 concurrent workers.


   [Breadth] RC阈值 t=5 完成 (簇数=6, d_cut=10)
   [Breadth] RC阈值 t=6 完成 (簇数=8, d_cut=9)
   [Breadth] RC阈值 t=7 完成 (簇数=9, d_cut=8)
   [Breadth] RC阈值 t=8 完成 (簇数=14, d_cut=7)
   [Breadth] RC阈值 t=9 完成 (簇数=18, d_cut=6)
   [Breadth] RC阈值 t=10 完成 (簇数=23, d_cut=5)
   [Breadth] RC阈值 t=11 完成 (簇数=29, d_cut=4)
   [Breadth] RC阈值 t=12 完成 (簇数=37, d_cut=3)
   [Breadth] RC阈值 t=13 完成 (簇数=41, d_cut=2)
   [Breadth] RC阈值 t=14 完成 (簇数=45, d_cut=1)
   [Breadth] RC阈值 t=15 完成 (簇数=49, d_cut=0)

 🚀 开始处理: Intensity
   📂 结果将保存至: f:\Desktop\科研项目\1.负责科研项目\Climate Policy\CAMPF_Supplementary\data\4-2-group_overlapping_cluster\Intensity
   正在计算聚类树（Complete Linkage）并检查重心...
   [Intensity 重心] 左: 8117 | 右: 7680
   >>> ✅ 重心正常。
   🎯 待处理阈值RC (11个): [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
      解释：每个 t 表示 Overlap ∈ (t-1, +∞]（等价 Overlap ≥ t）


[Parallel(n_jobs=-3)]: Done   3 out of  11 | elapsed:   21.9s remaining:   58.5s
[Parallel(n_jobs=-3)]: Done   6 out of  11 | elapsed:   24.8s remaining:   20.7s


   [Intensity] RC阈值 t=5 完成 (簇数=5, d_cut=10)
   [Intensity] RC阈值 t=6 完成 (簇数=8, d_cut=9)
   [Intensity] RC阈值 t=7 完成 (簇数=12, d_cut=8)
   [Intensity] RC阈值 t=8 完成 (簇数=14, d_cut=7)
   [Intensity] RC阈值 t=9 完成 (簇数=16, d_cut=6)
   [Intensity] RC阈值 t=10 完成 (簇数=19, d_cut=5)
   [Intensity] RC阈值 t=11 完成 (簇数=28, d_cut=4)
   [Intensity] RC阈值 t=12 完成 (簇数=35, d_cut=3)
   [Intensity] RC阈值 t=13 完成 (簇数=40, d_cut=2)
   [Intensity] RC阈值 t=14 完成 (簇数=46, d_cut=1)
   [Intensity] RC阈值 t=15 完成 (簇数=49, d_cut=0)

✅ 所有任务已完成！


[Parallel(n_jobs=-3)]: Done   9 out of  11 | elapsed:   25.2s remaining:    5.5s
[Parallel(n_jobs=-3)]: Done  11 out of  11 | elapsed:   25.3s finished


### 不同聚类的共识国家识别表格汇总

In [ ]:
import pandas as pd
from pathlib import Path
from scipy.cluster.hierarchy import fcluster, dendrogram as scipy_dendrogram

# 路径配置
current_dir = Path.cwd()
data_dir = current_dir.parent / "data"
mapping_output_path = data_dir / "4-2-Consensus_Policy_Cluster_Mapping.csv"
stats_output_path = data_dir / "4-2-Consensus_Cluster_Statistics.csv"

records = []
cluster_stats = []

# 核心逻辑：遍历所有K值生成映射
for k_value in valid_k_values:
    # 获取原始聚类顺序
    temp_dend = scipy_dendrogram(row_linkage, no_plot=True)
    original_order = temp_dend['leaves']
    
    # 反转顺序
    reversed_order = original_order[::-1]
    
    # 获取原始聚类标签（0-based）
    original_labels = fcluster(row_linkage, t=k_value, criterion='maxclust') - 1
    reordered_original_labels = [original_labels[i] for i in reversed_order]
    
    # 创建新的聚类ID映射：按反转后的顺序从1开始重新编号
    new_cluster_id = 1
    old_to_new_map = {}
    
    for old_label in reordered_original_labels:
        if old_label not in old_to_new_map:
            old_to_new_map[old_label] = new_cluster_id
            new_cluster_id += 1
    
    # 为每个国家分配新的聚类ID
    cluster_counts = {}
    for country, old_cluster_id in zip(countries, original_labels):
        new_cid = old_to_new_map[old_cluster_id]  # 使用映射后的ID
        records.append({"K值": k_value, "共识聚类ID": new_cid, "国家": country})
        cluster_counts[new_cid] = cluster_counts.get(new_cid, 0) + 1
    
    # 统计每个聚类的国家数
    for cid, count in sorted(cluster_counts.items()):
        cluster_stats.append({"K值": k_value, "共识聚类ID": cid, "国家数": count})

# 映射表
df_mapping = pd.DataFrame(records).sort_values(["K值", "共识聚类ID", "国家"]).reset_index(drop=True)
df_mapping.to_csv(mapping_output_path, index=False, encoding="utf-8-sig")

# 统计表
df_stats = pd.DataFrame(cluster_stats)
df_stats.to_csv(stats_output_path, index=False, encoding="utf-8-sig")

print(f"✓ 映射表已保存：{mapping_output_path}")
print(f"✓ 统计表已保存：{stats_output_path}")
print(f"总记录数：{len(df_mapping)} | K值范围：{valid_k_values}")
